In [7]:
import os
import re
import numpy as np
from glob import glob
import subprocess
os.environ['ALESPICEROOT'] = '/scratch/spice'

import spiceypy as spice

import ale
# from ale.base.data_naif import NaifSpice
# from ale.base.label_isis import IsisLabel
# from ale.base.type_sensor import Framer
# from ale.base.base import Driver
from ale.drivers.voyager_drivers import Voyager2IssnacIsisLabelNaifSpiceDriver
from ale import util

from ale.formatters.isis_formatter import to_isis

In [2]:
def get_image_kernels(image):
    """
    Get the kernels to use with a test image.

    Parameters
    ----------
    image : str
            The image name to get kernels for. I.E. 'EN1072174528M'

    Returns
    -------
    : list
      A list containing the absolute paths to the kernels for the images.
      This list contains all of the kernel files available in the test image's
      data directory which may contain transfer files that need to be converted
      to binary files.
    """
    if not isinstance(image, str):
        try:
            image = str(image)
        except:
            raise KeyError('Cannot coerce requested image name to string')
    if image in image_2_data:
        return image_2_data[image]
    else:
        raise KeyError('Could not find test data for' + image)
        
def convert_kernels(kernels):
    """
    Convert any transfer kernels in a list to binary kernels

    Parameters
    ----------
    kernels : list
              A list of kernels. Only transfer kernels present in the list will
              be converted. Non-transfer kernels will be ignored.

    Returns
    -------
    updated_kernels : list
                      The updated kernel list where all transfer kernels have
                      been replaced with their converted binary kernel. This
                      is designed to be passed directly to furnsh.
    binary_kernels : list
                     The list of binary kernels created.
    """
    binary_kernels = []
    updated_kernels = []
    for kernel in kernels:
        split_kernel = os.path.splitext(kernel)
        if 'x' in split_kernel[1].lower():
            # Get the full path to the kernel then truncate it to the relative path
            path = os.path.join(data_root, kernel)
            path = os.path.relpath(path)
            bin_output = subprocess.run(['tobin', path],
                                        capture_output=True, check=True)
            matches = re.search(r'To: (.*\.b\w*)', str(bin_output.stdout))
            if not matches:
                warnings.warn('Failed to convert transfer kernel, ' + kernel + ', skipping...')
            else:
                kernel = matches.group(1)
                binary_kernels.append(kernel)
        updated_kernels.append(kernel)
    return updated_kernels, binary_kernels

In [25]:
ale_root = os.path.split(ale.__file__)[0]
data_root = os.path.join(ale_root, '../tests/pytests/data')
dirs = next(os.walk(data_root, topdown=True))[1]
dirs = [d for d in dirs if not d.startswith('.')]
image_2_data = {}

for d in dirs:
    tmp = os.path.join(data_root, d)
    image_2_data[d] = [os.path.join(tmp, f) for f in os.listdir(tmp) if not f.startswith('.') and os.path.splitext(f)[1] != '.lbl']


kernels = get_image_kernels('c2065022')
updated_kernels, _ = convert_kernels(kernels)

In [32]:
cub = '/home/acpaquette/voyager/c2065022.cub'
spice_cub = '/home/acpaquette/voyager/c2065022.spice.cub'

# driver = Voyager2IssnacIsisLabelNaifSpiceDriver(cub)

with Voyager2IssnacIsisLabelNaifSpiceDriver(cub, props = {'kernels': updated_kernels}) as driver:
    print(to_isis(driver))
    frame_chain = driver.frame_chain
    target_frame_id = driver.target_frame_id
    sensor_frame_id = driver.sensor_frame_id
    positions, velocities, times = driver.sensor_position
#     print(driver.sensor_orientation)
    state, _ = spice.spkezr(driver.target_name,
                       -646346832.89712,
                       driver.reference_frame,
                       driver.light_time_correction,
                       driver.spacecraft_name,)
#     positions = [state[:3]]
#     velocities = [state[3:]]

[[ 0.08584925  0.69748692 -0.62461826  0.34057882]]
{'CameraVersion': 1, 'NaifKeywords': {'BODY502_RADII': array([1564.13, 1561.23, 1560.93]), 'BODY_FRAME_CODE': 10024, 'INS-32101_PIXEL_SIZE': 0.011789473651194, 'INS-32101_ITRANSL': [0.0, 0.0, 84.8214288089711], 'INS-32101_ITRANSS': [0.0, 84.8214288089711, 0.0], 'INS-32101_FOCAL_LENGTH': 1503.49, 'INS-32101_BORESIGHT_SAMPLE': 500.0, 'INS-32101_BORESIGHT_LINE': 500.0}, 'InstrumentPointing': {'TimeDependentFrames': [-32101, 1], 'CkTableStartTime': -646346830.7771244, 'CkTableEndTime': -646346830.7771244, 'CkTableOriginalSize': 1, 'EphemerisTimes': array([-6.46346831e+08]), 'Quaternions': array([[ 0.08584925,  0.69748692, -0.62461826,  0.34057882]])}, 'BodyRotation': {'TimeDependentFrames': [10024, 1], 'CkTableStartTime': -646346830.7771244, 'CkTableEndTime': -646346830.7771244, 'CkTableOriginalSize': 1, 'EphemerisTimes': array([-6.46346831e+08]), 'Quaternions': array([[-0.01010212, -0.22183773,  0.97458315,  0.02955773]])}, 'InstrumentPo

In [34]:
rotation = frame_chain.compute_rotation(target_frame_id, 1)
rotation._rots.apply(positions[0]) * -1/1000

array([[-133396.12418842, -184605.64812144,    3162.60650649]])

In [ ]:
133425.48293894,184605.07752753,-3162.2190909154

In [9]:
test_array = np.array([[1, 2, 3], [4, 5, 6]])

In [21]:
your_permutation = [1, 2, 0]

In [23]:
test_array

array([[2, 3, 1],
       [5, 6, 4]])

In [35]:
np.isclose(500, 500.0)

True